In [ ]:
import pandas as pd
import requests

## Setting up reddit api

#### If you wish to run this code snippet, follow the 'Getting Access' section on this link and then add your username and password below.
https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

In [219]:
CLIENT_ID = 'lUwEdnZWTKkfjs7uxgY00A'
SECRET_TOKEN = 'g0c8TflulWb8n4bNozY-l3veL0Tw4A'
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_TOKEN)

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': '',
        'password': ''}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'Group21/0.0.1'}
# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}
# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()

{'is_employee': False,
 'seen_layout_switch': False,
 'has_visited_new_profile': False,
 'pref_no_profanity': True,
 'has_external_account': False,
 'pref_geopopular': '',
 'seen_redesign_modal': False,
 'pref_show_trending': True,
 'subreddit': {'default_set': True,
  'user_is_contributor': False,
  'banner_img': '',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '',
  'user_is_muted': False,
  'display_name': 'u_aryan-veer',
  'header_img': None,
  'title': '',
  'coins': 0,
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://styles.redditmedia.com/t5_31nu5r/styles/profileIcon_snooca341a99-a2ea-433c-9217-945f61aae4ce-headshot-f.png?width=256&amp;height=256&amp;crop=256:256,smart&amp;s=a5f63dca859c1432fccc2be14d3c0cd9b0fb30da',
  'description': '',
  'submit_link_label': '',
  'header_size': None,
  'restrict_commenting':

In [311]:
def createdDfForRedditAPI(res, df):
    for post in res.json()['data']['children']:
        df = df.append({
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'subreddit': post['data']['subreddit']
        }, ignore_index=True)
    return df

def createdDfForPushshift(res, df):
    for post in res.json()['data']:
        try:
            df = df.append({
                'title': post['title'],
                'selftext': post['selftext'],
                'subreddit': post['subreddit']
            }, ignore_index=True)
        except KeyError:
            print(post)
    return df

## Jokes Dataframe

In [287]:
# Using the reddit API
df1 = pd.DataFrame() 

last_post = ''
last_post_id = ''
for i in range(0,10):
    if(i==0):
        post_res = requests.get('https://oauth.reddit.com/r/jokes/hot', headers=headers, params={'limit':'100'})
        if(len(post_res.json()['data']['children']) > 1):
            last_post = post_res.json()['data']['children'][-1]
            last_post_id = last_post['data']['name']
    else:
        post_res = requests.get('https://oauth.reddit.com/r/jokes/hot', headers=headers, params={'limit':'100', 'after':last_post_id})
        if(len(post_res.json()['data']['children']) > 1):
            last_post = post_res.json()['data']['children'][-1]
            last_post_id = last_post['data']['name']
    df1 = createdDfForRedditAPI(post_res, df1)
    
# Using the pushshift API and them merging this with reddit api data
df2 = pd.DataFrame()
for i in range(0,len(after)):
    res = requests.get("https://api.pushshift.io/reddit/search/submission/?subreddit=jokes&after=" + after[i] + "&before=" + before[i] + "&size=500")
    df2 = createdDfForPushshift(res, df2)
    
jokes_df = df2.append(df1, ignore_index=True)
jokes_df = jokes_df[jokes_df['selftext']!='[removed]']
jokes_df.drop_duplicates(inplace=True, ignore_index=True)
jokes_df.drop_duplicates('title', inplace=True)
jokes_df.drop_duplicates('selftext', inplace=True)

## DadJokes Dataframe

In [315]:
# Using the reddit API
df3 = pd.DataFrame() 

last_post = ''
last_post_id = ''
for i in range(0,10):
    if(i==0):
        post_res = requests.get('https://oauth.reddit.com/r/dadjokes/hot', headers=headers, params={'limit':'100'})
        if(len(post_res.json()['data']['children']) > 1):
            last_post = post_res.json()['data']['children'][-1]
            last_post_id = last_post['data']['name']
    else:
        post_res = requests.get('https://oauth.reddit.com/r/dadjokes/hot', headers=headers, params={'limit':'100', 'after':last_post_id})
        if(len(post_res.json()['data']['children']) > 1):
            last_post = post_res.json()['data']['children'][-1]
            last_post_id = last_post['data']['name']
    df3 = createdDfForRedditAPI(post_res, df3)
    
# date range from 15 Oct 2021 to 18 Nov 2021
after = ['1634301250', '1634560451', '1634819651', '1635078851', '1635338051', '1635597251', '1635942851', '1636202051', '1636461251', '1636720451', '1636979651']
before = ['1634560450', '1634819650', '1635078850', '1635338050', '1635597250', '1635942850', '1636202050', '1636461250', '1636720450', '1636979650', '1637238850']

# Using the pushshift API and them merging this with reddit api data
df4 = pd.DataFrame()
for i in range(0,len(after)):
    res = requests.get("https://api.pushshift.io/reddit/search/submission/?subreddit=dadjokes&after=" + after[i] + "&before=" + before[i] + "&size=500")
    df4 = createdDfForPushshift(res, df4)
    
dadjokes_df = df4.append(df3, ignore_index=True)
dadjokes_df = dadjokes_df[dadjokes_df['selftext']!='[removed]']
dadjokes_df.drop_duplicates(inplace=True, ignore_index=True)
dadjokes_df.drop_duplicates('title', inplace=True)
dadjokes_df.drop_duplicates('selftext', inplace=True)

{'all_awardings': [], 'allow_live_comments': False, 'author': '[deleted]', 'author_flair_background_color': '', 'author_flair_css_class': None, 'author_flair_text': None, 'author_flair_text_color': 'dark', 'author_is_blocked': False, 'awarders': [], 'banned_by': 'moderators', 'can_mod_post': False, 'contest_mode': False, 'created_utc': 1634309757, 'domain': 'self.dadjokes', 'full_link': 'https://www.reddit.com/r/dadjokes/comments/q8q4yi/when_you_separated_the_first_character_to_the/', 'gildings': {}, 'id': 'q8q4yi', 'is_created_from_ads_ui': False, 'is_crosspostable': False, 'is_meta': False, 'is_original_content': False, 'is_reddit_media_domain': False, 'is_robot_indexable': False, 'is_self': True, 'is_video': False, 'link_flair_background_color': '', 'link_flair_css_class': 'post-removed-punchline', 'link_flair_richtext': [], 'link_flair_text': ' Removed - Punchline in title ', 'link_flair_text_color': 'dark', 'link_flair_type': 'text', 'locked': False, 'media_only': False, 'no_follo

## Exporting dataframes to csv files

In [319]:
jokes_df.to_csv('jokes.csv', encoding='utf-8', index=False)
dadjokes_df.to_csv('dadjokes.csv', encoding='utf-8', index=False)